In [ ]:
import sqlite3

conn = sqlite3.connect("ingredients.db")
cursor = conn.cursor()

# 테이블 생성
cursor.execute("""
CREATE TABLE IF NOT EXISTS ingredients (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    parent_id INTEGER,
    FOREIGN KEY (parent_id) REFERENCES ingredients(id)
)
""")

# 데이터 삽입 (계층 구조)
ingredients = [
    (None, "육류"),
    (1, "돼지고기"),
    (1, "소고기"),
    (2, "삼겹살"),
    (2, "목살"),
    (3, "한우")
]

cursor.executemany("INSERT INTO ingredients (parent_id, name) VALUES (?, ?)", ingredients)

conn.commit()
conn.close()

print("데이터 삽입 완료!")


In [ ]:
def get_parent(ingredient_name):
    conn = sqlite3.connect("ingredients.db")
    cursor = conn.cursor()

    cursor.execute("""
    SELECT parent.name
    FROM ingredients AS child
    JOIN ingredients AS parent ON child.parent_id = parent.id
    WHERE child.name = ?
    """, (ingredient_name,))

    result = cursor.fetchone()
    conn.close()
    return result[0] if result else "최상위 카테고리"

print(get_parent("삼겹살"))  # 출력: 돼지고기


In [ ]:
def print_tree(parent_id=None, level=0):
    conn = sqlite3.connect("ingredients.db")
    cursor = conn.cursor()

    cursor.execute("""
    SELECT id, name FROM ingredients WHERE parent_id IS ?
    """, (parent_id,))

    results = cursor.fetchall()
    conn.close()

    for row in results:
        print(" " * (level * 4) + f"- {row[1]}")
        print_tree(row[0], level + 1)

print_tree()  # 트리 형태로 전체 출력


In [ ]:
def delete_ingredient(name):
    conn = sqlite3.connect("ingredients.db")
    cursor = conn.cursor()

    cursor.execute("DELETE FROM ingredients WHERE name = ?", (name,))
    conn.commit()

    if cursor.rowcount > 0:
        print(f"{name} 삭제 완료!")
    else:
        print(f"{name}을(를) 찾을 수 없습니다.")

    conn.close()

delete_ingredient("한우")

In [ ]:
import sqlite3

def add_ingredient(name, parent_name):
    conn = sqlite3.connect("ingredients.db")
    cursor = conn.cursor()

    # 중복 확인
    cursor.execute("SELECT id FROM ingredients WHERE name = ?", (name,))
    existing = cursor.fetchone()

    if existing:
        print(f"⚠️ '{name}'은(는) 이미 존재합니다.")
        conn.close()
        return

    # 부모 ID 찾기
    cursor.execute("SELECT id FROM ingredients WHERE name = ?", (parent_name,))
    parent_id = cursor.fetchone()

    if parent_id:
        parent_id = parent_id[0]
        cursor.execute("INSERT INTO ingredients (name, parent_id) VALUES (?, ?)", (name, parent_id))
        conn.commit()
        print(f"✅ '{name}' 추가 완료!")
    else:
        print(f"❌ 부모 카테고리 '{parent_name}'을(를) 찾을 수 없습니다.")

    conn.close()

# 예제 실행
add_ingredient("닭다리살", "닭고기")
add_ingredient("닭가슴살", "닭고기")
add_ingredient("닭안심살", "닭고기")

In [ ]:
import sqlite3

def add_parent_category(name):
    conn = sqlite3.connect("ingredients.db")
    cursor = conn.cursor()

    # 중복 확인
    cursor.execute("SELECT id FROM ingredients WHERE name = ?", (name,))
    existing = cursor.fetchone()

    if existing:
        print(f"⚠️ '{name}' 카테고리는 이미 존재합니다.")
    else:
        cursor.execute("INSERT INTO ingredients (name, parent_id) VALUES (?, NULL)", (name,))
        conn.commit()
        print(f"✅ 새로운 부모 카테고리 '{name}' 추가 완료!")

    conn.close()

# 예제 실행
add_parent_category("해산물")  # 새로운 상위 카테고리 추가
add_parent_category("채소")
add_parent_category("과일")
add_parent_category("조미료")
add_parent_category("견과류")
add_parent_category("향신료")

In [ ]:
import sqlite3

def add_ingredient(name, parent_name):
    conn = sqlite3.connect("ingredients.db")
    cursor = conn.cursor()

    # 부모 ID 찾기
    cursor.execute("SELECT id FROM ingredients WHERE name = ?", (parent_name,))
    parent_id = cursor.fetchone()

    if parent_id:
        parent_id = parent_id[0]

        # 중복 확인
        cursor.execute("SELECT id FROM ingredients WHERE name = ?", (name,))
        existing = cursor.fetchone()

        if existing:
            print(f"⚠️ '{name}'은(는) 이미 존재합니다.")
        else:
            cursor.execute("INSERT INTO ingredients (name, parent_id) VALUES (?, ?)", (name, parent_id))
            conn.commit()
            print(f"✅ '{name}' 추가 완료!")
    else:
        print(f"❌ 부모 카테고리 '{parent_name}'을(를) 찾을 수 없습니다.")

    conn.close()

# 실행 예제
add_ingredient("새우", "해산물")
add_ingredient("오징어", "해산물")
add_ingredient("닭고기", "육류")


